In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder \
.master("local[4]") \
.appName("DataCleaning") \
.config("spark.executor.memory","4g") \
.config("spark.driver.memory","2g") \
.getOrCreate()

# Veri Okuma

In [2]:
adult_train_df = spark.read \
.option("header","True") \
.option("inferSchema","True") \
.option("sep",",") \
.csv("D:\\Datasets\\adult.data") \

In [3]:
adult_test_df = spark.read \
.option("header","True") \
.option("inferSchema","True") \
.option("sep",",") \
.csv("D:\\Datasets\\adult.test") \

# Veri Birleştirme 

In [4]:
adult_whole_df = adult_train_df.union(adult_test_df)
adult_whole_df.limit(5).toPandas().head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,output
0,39,State-gov,77516.0,Bachelors,13.0,Never-married,Adm-clerical,Not-in-family,White,Male,2174.0,0.0,40.0,United-States,<=50K
1,50,Self-emp-not-inc,83311.0,Bachelors,13.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0.0,13.0,United-States,<=50K
2,38,Private,215646.0,HS-grad,9.0,Divorced,Handlers-cleaners,Not-in-family,White,Male,0.0,0.0,40.0,United-States,<=50K
3,53,Private,234721.0,11th,7.0,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0.0,0.0,40.0,United-States,<=50K
4,28,Private,338409.0,Bachelors,13.0,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0.0,0.0,40.0,Cuba,<=50K


/////////////////////// VERİ TEMİZLİĞİ İÇİN TAVSİYELER  ////////////////////////////////////////////////
    /*
      1. Tüm sütunları boşluk kontrolü yap.
      2. ? içeren workclass, occupation  var bunların ? içerdiği satırlar tekrar incelenmeli.
          ? işaretleri sistematik bir şekilde mi oluşmuş yoksa bu oluşum tesadüfi mi?
          ? kayıtlarının oluşması altında yatan bir mekanizma var mı?
          Bu sistematik hata yakalanırsa veri doldurma (imputation) yoksa satır silme yapılsın.
      3. workclass niteliğinde never-worked ve without-pay sınıfları ve
          occupation niteliğinde  Armed-Forces  sınıfı
        çok az tekrarlanmış. Veri setinden çıkarılabilir.
      4. education niteliğindeki:
              1st-4th, 5th-6th, 7th-8th: elementary-school
              9th, 10th, 11th, 12th: high-school
              Masters, Doctorate: high-education
              Bachelors, Some-college: undergraduate
         sınıfları yukarıdaki gibi birleştirilebilir.
      5. native_country'de ? var ve Hollanda 1 kez tekrarlanmış.
      6. output (hedef değişkendeki) >50K. ve <=50K. değerlerindeki "." kaldırılmalı
     */

# ====  VERİ TEMİZLİĞİ BAŞLIYOR ===========

### 1. Tüm sütunları boşluk kontrolü yap.

In [5]:
from pyspark.sql.functions import *

In [6]:
kategorik_nitelikler = ["workclass", "education", "marital_status", "occupation", "relationship", "race",
          "sex", "native_country", "output"]
numerik_nitelikler = ["age", "fnlwgt", "education_num", "capital_gain", "capital_loss", "hours_per_week"]


Boşluk temizleme (trim) sadece string değişkenlere uygulanabilir.

In [23]:
adult_whole_df1 = adult_whole_df \
.withColumn("workclass", trim(col("workclass"))) \
.withColumn("education", trim(col("education"))) \
.withColumn("marital_status", trim(col("marital_status"))) \
.withColumn("occupation", trim(col("occupation"))) \
.withColumn("relationship", trim(col("relationship"))) \
.withColumn("race", trim(col("race"))) \
.withColumn("sex", trim(col("sex"))) \
.withColumn("native_country", trim(col("native_country"))) \
.withColumn("output", trim(col("output")))

In [24]:
adult_whole_df.count()

48842

In [25]:
adult_whole_df1.count()

48842

#  2. OUTPUT İÇİNDEKİ "." TEMİZLİĞİ 
 Her ne kadar taslak planda output temizliğini son madde olarak yazmış olsak da. İşin kolay olması ve diğer temizlik
işlemlerini etkiliyor olması nedeniyle. Bu temizliği en başta yapalım.

    // . içerenleri filtreleyelim ancak "~" ile tersini yapalım.

In [35]:
adult_whole_df2 = adult_whole_df1 \
.withColumn("output", regexp_replace(col("output"), "<=50K.","<=50K")) \
.withColumn("output", regexp_replace(col("output"), ">50K.",">50K"))

In [36]:
# Temizlik sonucunu görelim.
adult_whole_df2.groupBy(col("output")).agg({"*":"count"}) \
.toPandas().head()


,output,count(1)
0,<=50K,37155
1,>50K,11687


# 3. NULL KONTROLÜ

    // Her bir sütun için for dongüsü ile o sütunda filter() metodu ile null kontrolü yapalım ve bunları sayalım. Eğer o sütunda en az bir adet null varsa yani >0 ise o sütunda null olduğunu ekrana yazdıralım.

In [38]:
adult_whole_df2.columns

['age',
 'workclass',
 'fnlwgt',
 'education',
 'education_num',
 'marital_status',
 'occupation',
 'relationship',
 'race',
 'sex',
 'capital_gain',
 'capital_loss',
 'hours_per_week',
 'native_country',
 'output']

In [66]:
sayac_for_null = 1
for sutun in adult_whole_df2.columns:
    if(adult_whole_df2.filter(col(sutun).isNull()).count() > 0):
        print(sayac_for_null, ". ", sutun, " içinde null değer var.")
    else:
        print(sayac_for_null, ". ", sutun)
    sayac_for_null+=1

1 .  age
2 .  workclass
3 .  fnlwgt
4 .  education
5 .  education_num
6 .  marital_status
7 .  occupation
8 .  relationship
9 .  race
10 .  sex
11 .  capital_gain
12 .  capital_loss
13 .  hours_per_week
14 .  native_country
15 .  output


    // Herhangi bir null yok.

# 4.  "?" KONTROLLERİ

In [69]:
sayac_for_question = 1
for sutun in adult_whole_df2.columns:
    if(adult_whole_df2.filter(col(sutun).contains("?")).count() > 0):
        print(sayac_for_question, ". ", sutun, " içinde ? var.")
    else:
        print(sayac_for_question, ". ", sutun)
    sayac_for_question+=1

1 .  age
2 .  workclass  içinde ? var.
3 .  fnlwgt
4 .  education
5 .  education_num
6 .  marital_status
7 .  occupation  içinde ? var.
8 .  relationship
9 .  race
10 .  sex
11 .  capital_gain
12 .  capital_loss
13 .  hours_per_week
14 .  native_country  içinde ? var.
15 .  output


    // workclass, occupation, native_country içinde ? var.
    
    // Bunların hedef değişken output ile ilişkisini inceleyelim.

In [53]:
adult_whole_df2.select("workclass", "occupation", "native_country", "output") \
.filter(col("workclass").contains("?") | col("occupation").contains("?") | col("native_country").contains("?")) \
.groupBy("workclass", "occupation", "native_country", "output").count() \
.orderBy(col("count").desc()) \
.toPandas().head(50)

,workclass,occupation,native_country,output,count
0,?,?,United-States,<=50K,2284
1,?,?,United-States,>50K,246
2,Private,Other-service,?,<=50K,100
3,Private,Sales,?,<=50K,55
4,Private,Prof-specialty,?,<=50K,51
5,Private,Craft-repair,?,<=50K,48
6,Private,Prof-specialty,?,>50K,48
7,?,?,Mexico,<=50K,48
8,Private,Adm-clerical,?,<=50K,47
9,Private,Machine-op-inspct,?,<=50K,42


    // Soru işaretlerinin dağılımı ve hedef değişken ile ilgisi tesadüfi görünüyor. Bu durumda ? işareti içeren satırları veri setinden çıkaralım.

In [54]:
adult_whole_df3 = adult_whole_df2 \
        .filter(~(col("workclass").contains("?") | col("occupation").contains("?") | col("native_country").contains("?")))

In [55]:
print(adult_whole_df2.count())
print(adult_whole_df3.count())

48842
45222


    // adult_whole_df2'den adult_whole_df3'e geçerken ? işareti olan satırlar silindi.

# 5. ZAYIF SINIFLARIN KALDIRILMASI

     // workclass niteliğinde never-worked ve without-pay sınıfları ve
    occupation niteliğinde  Armed-Forces  sınıfı
      çok az tekrarlanmış. Veri setinden çıkarılabilir.

In [56]:
adult_whole_df4 = adult_whole_df3 \
.filter(~(col("workclass").contains("never-worked") | col("workclass").contains("without-pay") |
          col("occupation").contains("Armed-Forces")))

In [57]:
print(adult_whole_df3.count())
print(adult_whole_df4.count())

45222
45208


    // adult_whole_df3'ten adult_whole_df4'e geçerken zayıf sınıfların bulunduğu satırlar silindi.

# 6. EĞİTİM DURUMUYLA İLGİLİ KATEGORİLERİ BİRLEŞTİRME

    // education niteliğindeki:
      1st-4th, 5th-6th, 7th-8th: elementary-school
    9th, 10th, 11th, 12th: high-school
    Masters, Doctorate: high-education
    Bachelors, Some-college: undergraduate
    sınıfları yukarıdaki gibi birleştirilebilir.

## Uyarı: Aşağıdaki yöntem trim edilmemiş niteliklerde kullanılmamalıdır.

In [58]:
adult_whole_df5 = adult_whole_df4.withColumn("education_merged", \
      when(col("education").isin("1st-4th","education","5th-6th","education","7th-8th"), "Elementary-School")
        .when(col("education").isin("9th","10th","11th","12th"), "High-School")
        .when(col("education").isin("Masters","Doctorate"), "Post-Graduate")
        .when(col("education").isin("Bachelors","Some-college"), "Under-Graduate")
        .otherwise(col("education")))
adult_whole_df5.select("education","education_merged").toPandas().head(20)

,education,education_merged
0,Bachelors,Under-Graduate
1,Bachelors,Under-Graduate
2,HS-grad,HS-grad
3,11th,High-School
4,Bachelors,Under-Graduate
5,Masters,Post-Graduate
6,9th,High-School
7,HS-grad,HS-grad
8,Masters,Post-Graduate
9,Bachelors,Under-Graduate


In [60]:
adult_whole_df5.groupBy("education_merged").agg({"*":"count"}).toPandas().head(20)

,education_merged,count(1)
0,Assoc-acdm,1507
1,Assoc-voc,1959
2,Elementary-School,1494
3,High-School,4094
4,HS-grad,14778
5,Preschool,72
6,Post-Graduate,3056
7,Under-Graduate,17464
8,Prof-school,784


    // Evet yeni bir sütun ekledik. education_merged. Daha sade.

# 7. native_country niteliğinde temizlik

    // native_country'de ? var ve Hollanda 1 kez tekrarlanmış.
    // ? temizleme işini 4. aşamada hallettik.
    // Burada sadece Hollandayı (Holand-Netherlands) çıkaralım.

In [61]:
adult_whole_df6 = adult_whole_df5.filter(~(col("native_country") == "Holand-Netherlands"))

In [62]:
print(adult_whole_df5.count())
print(adult_whole_df6.count())

45208
45207


Satır bir azaldığına göre çıkarma işlemi başarılı.

# 8. OUTPUT . KALDIRMA

     // Bu temizliği 2. aşamada yapmıştık.

# 9. TEMİZ VERİYİ DİSKE YAZALIM

    // Sütun sırasını değiştirelim. education_merge'i education yanına alalım

In [63]:
nitelik_siralama = ["workclass", "education", "education_merged", "marital_status", "occupation", "relationship", "race",
     "sex", "native_country", "age", "fnlwgt", "education_num", "capital_gain", "capital_loss", "hours_per_week","output"]
adult_whole_df7 = adult_whole_df6.select(nitelik_siralama)

In [64]:
adult_whole_df7.toPandas().head()

,workclass,education,education_merged,marital_status,occupation,relationship,race,sex,native_country,age,fnlwgt,education_num,capital_gain,capital_loss,hours_per_week,output
0,State-gov,Bachelors,Under-Graduate,Never-married,Adm-clerical,Not-in-family,White,Male,United-States,39,77516.0,13.0,2174.0,0.0,40.0,<=50K
1,Self-emp-not-inc,Bachelors,Under-Graduate,Married-civ-spouse,Exec-managerial,Husband,White,Male,United-States,50,83311.0,13.0,0.0,0.0,13.0,<=50K
2,Private,HS-grad,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,United-States,38,215646.0,9.0,0.0,0.0,40.0,<=50K
3,Private,11th,High-School,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,United-States,53,234721.0,7.0,0.0,0.0,40.0,<=50K
4,Private,Bachelors,Under-Graduate,Married-civ-spouse,Prof-specialty,Wife,Black,Female,Cuba,28,338409.0,13.0,0.0,0.0,40.0,<=50K


In [65]:
 adult_whole_df7 \
      .coalesce(1) \
      .write \
      .mode("overwrite") \
      .option("sep",",") \
      .option("header","true") \
      .csv("D:\\Datasets\\adult_preprocessed_pyspark")